[LO 1, LO 2 – 30 Poin] Buatlah model machine learning yang berisi proses pre-processing machine 
learning, training, dan hasil dari perbadingan 2 algoritma machine learning yaitu Random Forest dan 
Xgboost. Ambil algoritma terbaik dan simpan dengan menggunakan pickle. Seluruh proses disimpan 
dengan extension .ipynb

In [22]:
# 1. Import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score
from scipy.stats import zscore
import warnings
warnings.filterwarnings("ignore")


In [ ]:
# membaca dataset 
df = pd.read_csv("Dataset_A_loan.csv")
df.head()

,person_age,person_gender,person_education,person_income,person_emp_exp,person_home_ownership,loan_amnt,loan_intent,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,previous_loan_defaults_on_file,loan_status
0,22.0,female,Master,71948.0,0,RENT,35000.0,PERSONAL,16.02,0.49,3.0,561,No,1
1,21.0,female,High School,12282.0,0,OWN,1000.0,EDUCATION,11.14,0.08,2.0,504,Yes,0
2,25.0,female,High School,12438.0,3,MORTGAGE,5500.0,MEDICAL,12.87,0.44,3.0,635,No,1
3,23.0,female,Bachelor,79753.0,0,RENT,35000.0,MEDICAL,15.23,0.44,2.0,675,No,1
4,24.0,male,Master,66135.0,1,RENT,35000.0,MEDICAL,14.27,0.53,4.0,586,No,1


In [ ]:
#melihat  overview dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45000 entries, 0 to 44999
Data columns (total 14 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   person_age                      45000 non-null  float64
 1   person_gender                   45000 non-null  object 
 2   person_education                45000 non-null  object 
 3   person_income                   42750 non-null  float64
 4   person_emp_exp                  45000 non-null  int64  
 5   person_home_ownership           45000 non-null  object 
 6   loan_amnt                       45000 non-null  float64
 7   loan_intent                     45000 non-null  object 
 8   loan_int_rate                   45000 non-null  float64
 9   loan_percent_income             45000 non-null  float64
 10  cb_person_cred_hist_length      45000 non-null  float64
 11  credit_score                    45000 non-null  int64  
 12  previous_loan_defaults_on_file  

In [ ]:
# Print semua kolom unique value 
for column in df.columns:
    print(f"Unique values in '{column}':")
    print(df[column].unique())
    print("-" * 40)

Unique values in 'person_age':
[ 22.  21.  25.  23.  24.  26. 144. 123.  20.  32.  34.  29.  33.  28.
  35.  31.  27.  30.  36.  40.  50.  45.  37.  39.  44.  43.  41.  46.
  38.  47.  42.  48.  49.  58.  65.  51.  53.  66.  61.  54.  57.  59.
  62.  60.  55.  52.  64.  70.  78.  69.  56.  73.  63.  94.  80.  84.
  76.  67. 116. 109.]
----------------------------------------
Unique values in 'person_gender':
['female' 'male' 'Male' 'fe male']
----------------------------------------
Unique values in 'person_education':
['Master' 'High School' 'Bachelor' 'Associate' 'Doctorate']
----------------------------------------
Unique values in 'person_income':
[71948. 12282. 12438. ... 31924. 56942. 33164.]
----------------------------------------
Unique values in 'person_emp_exp':
[  0   3   1   5   4   2   7   6 125   8 121 101 100  12  10   9  14  13
  11  15  16  17  19  28  25  18  24  22  20  23  21  31  26  27  29  32
  30 124  40  43  33  44  34  42  37  45  36  41  47  38  39  35  57  

In [26]:
# 3. Preprocessing

# Imputasi nilai yang hilang
df['person_income'].fillna(df['person_income'].median(), inplace=True)


In [27]:
# Normalisasi gender
df['person_gender'] = df['person_gender'].str.lower().str.replace(' ', '')  # jadi 'female', 'male', 'female' (dari 'fe male')

In [28]:
# Drop duplikat
df.drop_duplicates(inplace=True)

In [29]:
# Hilangkan outlier menggunakan Z-score
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns
z_scores = np.abs(zscore(df[numerical_cols]))
df = df[(z_scores < 3).all(axis=1)]

In [30]:
# Label encoding kolom kategorikal
categorical_cols = df.select_dtypes(include=['object']).columns
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le


In [31]:
# Pisahkan fitur dan target
X = df.drop('loan_status', axis=1)
y = df['loan_status']

In [32]:
# Standardisasi fitur numerik
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [33]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
# Grid Search untuk Random Forest
param_grid = {
    'n_estimators': [50,100,150, 200,250],
    'max_depth': [5, 10, 15,20 ,None],
    'min_samples_split': [2, 3,4,5, 10],
    'min_samples_leaf': [1, 2,3,4,5]
}

rf = RandomForestClassifier(random_state=42)
grid_rf = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, scoring='accuracy', verbose=1, n_jobs=-1)
grid_rf.fit(X_train, y_train)

print("Random Forest Best Params:", grid_rf.best_params_)
rf_best = grid_rf.best_estimator_
rf_pred = rf_best.predict(X_test)

print("Random Forest Accuracy:", accuracy_score(y_test, rf_pred))
print(classification_report(y_test, rf_pred))


Fitting 3 folds for each of 625 candidates, totalling 1875 fits
Random Forest Best Params: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 3, 'n_estimators': 150}
Random Forest Accuracy: 0.9204119850187266
              precision    recall  f1-score   support

           0       0.93      0.97      0.95      6703
           1       0.86      0.75      0.80      1841

    accuracy                           0.92      8544
   macro avg       0.90      0.86      0.88      8544
weighted avg       0.92      0.92      0.92      8544



In [ ]:
# Grid Search untuk XGBoost
grid_xgb = {
    'n_estimators': [50,100, 150,200,250],
    'max_depth': [3, 6, 10,15 ],
    'learning_rate': [0.01, 0.05, 0.1, 0.5,1],
    'subsample': [0.75, 0.8, 1.0]
}

xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
grid_xgb = GridSearchCV(estimator=xgb, param_grid=grid_xgb, cv=3, scoring='accuracy', verbose=1, n_jobs=-1)
grid_xgb.fit(X_train, y_train)

print("XGBoost Best Params:", grid_xgb.best_params_)
xgb_best = grid_xgb.best_estimator_
xgb_pred = xgb_best.predict(X_test)

print("XGBoost Accuracy:", accuracy_score(y_test, xgb_pred))
print(classification_report(y_test, xgb_pred))


Fitting 3 folds for each of 300 candidates, totalling 900 fits


XGBoost Best Params: {'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 200, 'subsample': 0.8}
XGBoost Accuracy: 0.9286048689138576
              precision    recall  f1-score   support

           0       0.94      0.97      0.96      6703
           1       0.88      0.77      0.82      1841

    accuracy                           0.93      8544
   macro avg       0.91      0.87      0.89      8544
weighted avg       0.93      0.93      0.93      8544



In [36]:
# Bandingkan akurasi
acc_rf = accuracy_score(y_test, rf_pred)
acc_xgb = accuracy_score(y_test, xgb_pred)

if acc_xgb > acc_rf:
    best_model = xgb_best
    best_name = "XGBoost"
else:
    best_model = rf_best
    best_name = "RandomForest"

print(f"\nModel terbaik adalah {best_name} dengan akurasi {max(acc_rf, acc_xgb)}")



Model terbaik adalah XGBoost dengan akurasi 0.9286048689138576


In [ ]:
#menyimpan model terbaik ke dalam pickle
import pickle

with open("best_model.pkl", "wb") as f:
    pickle.dump(best_model, f)
